In [ ]:
!pip install azure-ai-textanalytics==5.2.0b1
!pip install bs4
!pip install requests

In [5]:
# authenticate azure text analytics client
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

key = "677e7a68d2f94932a97572bfd9505ea3"
endpoint = "https://covid-corruption-nlp2.cognitiveservices.azure.com/"

def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()


In [ ]:
# perform key phrase extraction
def key_phrase_extraction(client, documents=""):
    try:
        responses = client.extract_key_phrases(documents = documents)
        
        key_phrases = []
        for response in responses:
            if response.key_phrases:
                key_phrases += response.key_phrases
        
        return key_phrases

    except Exception as err:
        print("Encountered exception. {}".format(err))


news_source = "sky"
f_in = open('datasets/parsed-news/{}.txt'.format(news_source), 'r')
documents = [line.rstrip() for line in f_in.readlines()]
num_docs = len(documents)

key_phrases = []
for i in range(0, num_docs, 10):
    print(i)
    key_phrase = key_phrase_extraction(
                    client, 
                    documents=documents[i: min(num_docs, i+10)]
    )

    if key_phrase:
        key_phrases += key_phrase

print(key_phrases)
f_out = open('datasets/parsed-news/{}_phrases.txt'.format(news_source), 'w+')
for phrase in key_phrases:
    phrase = '-'.join(phrase.split())
    f_out.write(phrase + '\n')

In [ ]:
# opinion mining on trustpilot reviews
from bs4 import BeautifulSoup
import requests
import pandas as pd

grouped_details = pd.read_csv('datasets/grouped_details.csv')
link_cols = [x for x in grouped_details.columns if 'company_link' in x]

def get_reviews(x):
    results = []
    for cell in x:
        if pd.isna(cell) or not cell:
            results.append('')
        else:
            page = requests.get("https://uk.trustpilot.com/review/"+cell)
            soup = BeautifulSoup(page.content, "html.parser")
            reviewcontent = soup.find_all('p', {'class', 'review-content__text'})
            print(page)
            reviews = []
            for review in reviewcontent:
                if not review:
                    reviews.append('')
                else:
                    reviews.append(review.get_text())
            results.append(reviews)
    return results

link_details = pd.DataFrame(grouped_details[link_cols])
tr_reviews = link_details.apply(get_reviews)



In [62]:
review_details = pd.DataFrame(grouped_details['company_name'])
review_details['company_link'] = grouped_details['company_link_0']
for col in link_cols: review_details['review_text'+col] = tr_reviews[col]
review_details.to_csv('text_test.csv')
tr_reviews.to_csv('review_test.csv')

In [ ]:
def opinion_mining(client,documents):

    result = client.analyze_sentiment(documents, show_opinion_mining=True)
    doc_result = [doc for doc in result if not doc.is_error]

    negative_keywords=[]
    for document in doc_result:
        for sentence in document.sentences:
            for mined_opinion in sentence.mined_opinions:
                target = mined_opinion.target
                if target.sentiment == 'negative':
                    negative_keywords.append(target.text)
                else:
                    negative_keywords.append('')
    return(negative_keywords)

In [45]:
print(tr_reviews['company_link_0'][0][1])
#opinion_mining(client,tr_reviews['company_link_0'][0])


                Family member used this company after returning from a amber country. They use oncologica which has shocking reviews.Day 6 and still no sign of the day 2 results.   They refuse to recognise the Royal Mail delivery time and send a email when they scan the item. Which could be up to 12 hrs later. This gives them even longer to process the test. Still no results. False advertising, trading standards and breach of contract!
            
